In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 200)
%matplotlib inline

# 训练集

In [26]:
train = pd.read_csv('train_pre.csv')

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743974 entries, 0 to 743973
Data columns (total 32 columns):
service_type              743974 non-null int64
is_mix_service            743974 non-null int64
online_time               743974 non-null int64
1_total_fee               743974 non-null float64
2_total_fee               743974 non-null float64
3_total_fee               743974 non-null float64
4_total_fee               743974 non-null float64
month_traffic             743974 non-null float64
many_over_bill            743974 non-null int64
contract_type             743974 non-null int64
contract_time             743974 non-null int64
is_promise_low_consume    743974 non-null int64
net_service               743974 non-null int64
pay_times                 743974 non-null int64
pay_num                   743974 non-null float64
last_month_traffic        743974 non-null float64
local_trafffic_month      743974 non-null float64
local_caller_time         743974 non-null float64
service

In [28]:
# train['gender'] = train['gender'].astype(int)
# train['age'] = train['age'].astype(int)

In [29]:
train_id = train[['user_id']]
train_y = train[['current_service']]
train_x = train.drop(['user_id', 'current_service'], axis=1)
train_feature = train_x.columns

In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_y_le = le.fit_transform(train_y['current_service'])

In [31]:
test = pd.read_csv('test_pre.csv')

In [32]:
test_id = test[['user_id']]
test_x = test.drop(['user_id'], axis=1)

## LGB程序

In [33]:
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(11, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='macro')
    return 'f1_score', score_vali**2, True

In [34]:
params = {'learning_rate':0.1, 
          'lambda_l1':0.2,
          'lambda_l2':0.25,
#           'max_depth':8,
          'num_leaves':78, 
          'metric':None, 
          'objective':'multiclass',
          'num_class':11, 
          'silent':True
          }

fold = 10 效果比5好一些 不过没有好太多

In [35]:
n_fold = 5
x_score = []
cv_pred = []
test_pred = []

In [36]:
skf = StratifiedKFold(n_splits=n_fold, random_state=0, shuffle=True)

In [38]:
from sklearn.externals import joblib

已经试过的降分的特征：fee_var 12_fee

fee_max 0.7408 fee_mean 0.74086 fee_online 0.741499 fee_diff 0.741107 net_contract 0.74147 fee_min 0.7408 fee_out 0.74307

ing:去掉了2行 fee_online

In [41]:
lgb_oof_train = np.zeros((len(train_x),))
for index, (train_index, valid_index) in enumerate(skf.split(train_x, train_y_le)):
    print(index)
    x_train, x_valid, y_train, y_valid = train_x.loc[train_index], train_x.loc[valid_index], train_y_le[train_index], train_y_le[valid_index]
    train_data = lgb.Dataset(x_train, label=y_train)
    valid_data = lgb.Dataset(x_valid, label=y_valid)
    clf = lgb.train(params, train_data, num_boost_round=100000, valid_sets=[valid_data], feval=f1_score_vali, early_stopping_rounds=50, verbose_eval=50)
    x_pred = clf.predict(x_valid, num_iteration=clf.best_iteration)
    x_pred = [list(x).index(max(x)) for x in x_pred]
    y_test = clf.predict(test_x, num_iteration=clf.best_iteration)
    y_test = [list(x).index(max(x)) for x in y_test]
    x_score.append(f1_score(y_valid,x_pred,average='macro'))

    lgb_oof_train[valid_index] = x_pred
    joblib.dump(clf, 'lgb'+str(index)+'origin.m')
    
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

0
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 0.232247	valid_0's f1_score: 0.773449
[100]	valid_0's multi_logloss: 0.192324	valid_0's f1_score: 0.792693
[150]	valid_0's multi_logloss: 0.180987	valid_0's f1_score: 0.80254
[200]	valid_0's multi_logloss: 0.175011	valid_0's f1_score: 0.808732
[250]	valid_0's multi_logloss: 0.170804	valid_0's f1_score: 0.81419
[300]	valid_0's multi_logloss: 0.1678	valid_0's f1_score: 0.817793
[350]	valid_0's multi_logloss: 0.165351	valid_0's f1_score: 0.821536
[400]	valid_0's multi_logloss: 0.163334	valid_0's f1_score: 0.824918
[450]	valid_0's multi_logloss: 0.161612	valid_0's f1_score: 0.827557
[500]	valid_0's multi_logloss: 0.160074	valid_0's f1_score: 0.8305
[550]	valid_0's multi_logloss: 0.158804	valid_0's f1_score: 0.832242
[600]	valid_0's multi_logloss: 0.157802	valid_0's f1_score: 0.834072
[650]	valid_0's multi_logloss: 0.156822	valid_0's f1_score: 0.835872
[700]	valid_0's multi_logloss: 0.155911	valid_

In [237]:
submit_lgb = []
for line in cv_pred:
    submit_lgb.append(np.argmax(np.bincount(line)))

In [191]:
submit_lgb = le.inverse_transform(submit_lgb)
df_test = test_id[['user_id']]
df_test['predict_service'] = submit_lgb

In [192]:
df_test.to_csv('lgb_vote.csv', index=None)

## XGB程序

In [238]:
import xgboost as xgb

def XGB():
    clf = xgb.XGBClassifier(max_depth=12, learning_rate=0.1,
                            n_estimators=500, silent=True,
                            objective="multi:softmax",
                            nthread=4, gamma=0,
                            max_delta_step=0, subsample=1, colsample_bytree=0.9, colsample_bylevel=0.9,
                            reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
                            base_score=0.5, seed=2018, missing=None,num_class=11)
    return clf

fee_min 0.7422

In [239]:
cv_pred = []
xgb_oof_train = np.zeros((len(train_x),))
for index, (train_index, valid_index) in enumerate(skf.split(train_x, train_y_le)):
    print(index)
    x_train, x_valid, y_train, y_valid = train_x.loc[train_index], train_x.loc[valid_index], train_y_le[train_index], train_y_le[valid_index]
    clf = XGB()
    clf.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=1)
    x_pred = clf.predict(x_valid)
#     x_pred = [list(x).index(max(x)) for x in x_pred]
#     x_score.append(clf.best_score['valid_0']['multi_logloss'])
    y_test = clf.predict(test_x)
    xgb_oof_train[valid_index] = x_pred
    
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

0
[0]	validation_0-merror:0.091396
[1]	validation_0-merror:0.085274
[2]	validation_0-merror:0.081699
[3]	validation_0-merror:0.079777
[4]	validation_0-merror:0.079037
[5]	validation_0-merror:0.077989
[6]	validation_0-merror:0.076786
[7]	validation_0-merror:0.076316
[8]	validation_0-merror:0.075697
[9]	validation_0-merror:0.0752
[10]	validation_0-merror:0.074797
[11]	validation_0-merror:0.074347
[12]	validation_0-merror:0.073977
[13]	validation_0-merror:0.073466
[14]	validation_0-merror:0.073103
[15]	validation_0-merror:0.072821
[16]	validation_0-merror:0.072283
[17]	validation_0-merror:0.071827
[18]	validation_0-merror:0.071625
[19]	validation_0-merror:0.071202
[20]	validation_0-merror:0.070825
[21]	validation_0-merror:0.070536
[22]	validation_0-merror:0.070395
[23]	validation_0-merror:0.070126
[24]	validation_0-merror:0.069972
[25]	validation_0-merror:0.069609
[26]	validation_0-merror:0.069273
[27]	validation_0-merror:0.069085
[28]	validation_0-merror:0.068789
[29]	validation_0-merror

[238]	validation_0-merror:0.052197
[239]	validation_0-merror:0.052129
[240]	validation_0-merror:0.052149
[241]	validation_0-merror:0.052096
[242]	validation_0-merror:0.052116
[243]	validation_0-merror:0.05219
[244]	validation_0-merror:0.052197
[245]	validation_0-merror:0.052149
[246]	validation_0-merror:0.052149
[247]	validation_0-merror:0.052129
[248]	validation_0-merror:0.052096
[249]	validation_0-merror:0.052163
[250]	validation_0-merror:0.052089
[251]	validation_0-merror:0.052049
[252]	validation_0-merror:0.052055
[253]	validation_0-merror:0.052089
[254]	validation_0-merror:0.052015
[255]	validation_0-merror:0.051955
[256]	validation_0-merror:0.051961
[257]	validation_0-merror:0.052022
[258]	validation_0-merror:0.052035
[259]	validation_0-merror:0.051961
[260]	validation_0-merror:0.051861
[261]	validation_0-merror:0.051813
[262]	validation_0-merror:0.0518
[263]	validation_0-merror:0.051834
[264]	validation_0-merror:0.051813
[265]	validation_0-merror:0.051827
[266]	validation_0-merr

[474]	validation_0-merror:0.050577
[475]	validation_0-merror:0.05055
[476]	validation_0-merror:0.05055
[477]	validation_0-merror:0.050523
[478]	validation_0-merror:0.050543
[479]	validation_0-merror:0.05053
[480]	validation_0-merror:0.05059
[481]	validation_0-merror:0.050577
[482]	validation_0-merror:0.05055
[483]	validation_0-merror:0.05051
[484]	validation_0-merror:0.05055
[485]	validation_0-merror:0.050503
[486]	validation_0-merror:0.050537
[487]	validation_0-merror:0.050584
[488]	validation_0-merror:0.05059
[489]	validation_0-merror:0.050597
[490]	validation_0-merror:0.050577
[491]	validation_0-merror:0.050604
[492]	validation_0-merror:0.050557
[493]	validation_0-merror:0.050557
[494]	validation_0-merror:0.05053
[495]	validation_0-merror:0.050543
[496]	validation_0-merror:0.050537
[497]	validation_0-merror:0.050516
[498]	validation_0-merror:0.050456
[499]	validation_0-merror:0.050456
1
[0]	validation_0-merror:0.091761
[1]	validation_0-merror:0.085625
[2]	validation_0-merror:0.08295

[212]	validation_0-merror:0.053273
[213]	validation_0-merror:0.053327
[214]	validation_0-merror:0.053152
[215]	validation_0-merror:0.053172
[216]	validation_0-merror:0.053112
[217]	validation_0-merror:0.053065
[218]	validation_0-merror:0.052964
[219]	validation_0-merror:0.052944
[220]	validation_0-merror:0.052944
[221]	validation_0-merror:0.05291
[222]	validation_0-merror:0.052856
[223]	validation_0-merror:0.05287
[224]	validation_0-merror:0.052749
[225]	validation_0-merror:0.052762
[226]	validation_0-merror:0.052728
[227]	validation_0-merror:0.052702
[228]	validation_0-merror:0.052715
[229]	validation_0-merror:0.052708
[230]	validation_0-merror:0.052628
[231]	validation_0-merror:0.05256
[232]	validation_0-merror:0.052587
[233]	validation_0-merror:0.0525
[234]	validation_0-merror:0.05252
[235]	validation_0-merror:0.05246
[236]	validation_0-merror:0.052386
[237]	validation_0-merror:0.052359
[238]	validation_0-merror:0.052312
[239]	validation_0-merror:0.052399
[240]	validation_0-merror:0

[447]	validation_0-merror:0.050625
[448]	validation_0-merror:0.050706
[449]	validation_0-merror:0.050733
[450]	validation_0-merror:0.050679
[451]	validation_0-merror:0.050692
[452]	validation_0-merror:0.050712
[453]	validation_0-merror:0.050672
[454]	validation_0-merror:0.050719
[455]	validation_0-merror:0.050706
[456]	validation_0-merror:0.050726
[457]	validation_0-merror:0.050699
[458]	validation_0-merror:0.050746
[459]	validation_0-merror:0.050726
[460]	validation_0-merror:0.050719
[461]	validation_0-merror:0.050726
[462]	validation_0-merror:0.050645
[463]	validation_0-merror:0.050645
[464]	validation_0-merror:0.050699
[465]	validation_0-merror:0.050672
[466]	validation_0-merror:0.050679
[467]	validation_0-merror:0.050699
[468]	validation_0-merror:0.050706
[469]	validation_0-merror:0.050719
[470]	validation_0-merror:0.050679
[471]	validation_0-merror:0.050699
[472]	validation_0-merror:0.050733
[473]	validation_0-merror:0.050773
[474]	validation_0-merror:0.050746
[475]	validation_0-m

...

[421]	validation_0-merror:0.051957
[422]	validation_0-merror:0.05201
[423]	validation_0-merror:0.051997
[424]	validation_0-merror:0.051963
[425]	validation_0-merror:0.051916
[426]	validation_0-merror:0.051923
[427]	validation_0-merror:0.051977
[428]	validation_0-merror:0.051957
[429]	validation_0-merror:0.051997
[430]	validation_0-merror:0.052017
[431]	validation_0-merror:0.051997
[432]	validation_0-merror:0.052004
[433]	validation_0-merror:0.052024
[434]	validation_0-merror:0.051984
[435]	validation_0-merror:0.052004
[436]	validation_0-merror:0.051963
[437]	validation_0-merror:0.052004
[438]	validation_0-merror:0.05197
[439]	validation_0-merror:0.051943
[440]	validation_0-merror:0.05193
[441]	validation_0-merror:0.051997
[442]	validation_0-merror:0.05195
[443]	validation_0-merror:0.051957
[444]	validation_0-merror:0.05197
[445]	validation_0-merror:0.05191
[446]	validation_0-merror:0.051923
[447]	validation_0-merror:0.051923
[448]	validation_0-merror:0.051943
[449]	validation_0-merror:

[160]	validation_0-merror:0.053885
[161]	validation_0-merror:0.053933
[162]	validation_0-merror:0.053818
[163]	validation_0-merror:0.053845
[164]	validation_0-merror:0.053765
[165]	validation_0-merror:0.053751
[166]	validation_0-merror:0.053717
[167]	validation_0-merror:0.05367
[168]	validation_0-merror:0.053543
[169]	validation_0-merror:0.053536
[170]	validation_0-merror:0.053502
[171]	validation_0-merror:0.05326
[172]	validation_0-merror:0.053334
[173]	validation_0-merror:0.053267
[174]	validation_0-merror:0.053254
[175]	validation_0-merror:0.053126
[176]	validation_0-merror:0.053126
[177]	validation_0-merror:0.053126
[178]	validation_0-merror:0.053119
[179]	validation_0-merror:0.053099
[180]	validation_0-merror:0.053106
[181]	validation_0-merror:0.053079
[182]	validation_0-merror:0.053133
[183]	validation_0-merror:0.053119
[184]	validation_0-merror:0.053066
[185]	validation_0-merror:0.053019
[186]	validation_0-merror:0.052985
[187]	validation_0-merror:0.053019
[188]	validation_0-mer

[395]	validation_0-merror:0.049941
[396]	validation_0-merror:0.049927
[397]	validation_0-merror:0.049846
[398]	validation_0-merror:0.04984
[399]	validation_0-merror:0.04984
[400]	validation_0-merror:0.049779
[401]	validation_0-merror:0.04984
[402]	validation_0-merror:0.049799
[403]	validation_0-merror:0.04986
[404]	validation_0-merror:0.049867
[405]	validation_0-merror:0.049867
[406]	validation_0-merror:0.049853
[407]	validation_0-merror:0.049941
[408]	validation_0-merror:0.049954
[409]	validation_0-merror:0.049981
[410]	validation_0-merror:0.049947
[411]	validation_0-merror:0.049974
[412]	validation_0-merror:0.050035
[413]	validation_0-merror:0.049994
[414]	validation_0-merror:0.050055
[415]	validation_0-merror:0.050014
[416]	validation_0-merror:0.050028
[417]	validation_0-merror:0.050061
[418]	validation_0-merror:0.050082
[419]	validation_0-merror:0.050061
[420]	validation_0-merror:0.050041
[421]	validation_0-merror:0.050035
[422]	validation_0-merror:0.049988
[423]	validation_0-merro

[133]	validation_0-merror:0.057913
[134]	validation_0-merror:0.057738
[135]	validation_0-merror:0.057677
[136]	validation_0-merror:0.057556
[137]	validation_0-merror:0.057496
[138]	validation_0-merror:0.057516
[139]	validation_0-merror:0.057422
[140]	validation_0-merror:0.057415
[141]	validation_0-merror:0.057288
[142]	validation_0-merror:0.057288
[143]	validation_0-merror:0.057207
[144]	validation_0-merror:0.057147
[145]	validation_0-merror:0.057133
[146]	validation_0-merror:0.057099
[147]	validation_0-merror:0.056992
[148]	validation_0-merror:0.056945
[149]	validation_0-merror:0.056945
[150]	validation_0-merror:0.05681
[151]	validation_0-merror:0.056797
[152]	validation_0-merror:0.056723
[153]	validation_0-merror:0.056589
[154]	validation_0-merror:0.056582
[155]	validation_0-merror:0.056589
[156]	validation_0-merror:0.056421
[157]	validation_0-merror:0.0563
[158]	validation_0-merror:0.056273
[159]	validation_0-merror:0.056259
[160]	validation_0-merror:0.056199
[161]	validation_0-merr

[368]	validation_0-merror:0.051938
[369]	validation_0-merror:0.051965
[370]	validation_0-merror:0.051951
[371]	validation_0-merror:0.051945
[372]	validation_0-merror:0.051965
[373]	validation_0-merror:0.051951
[374]	validation_0-merror:0.052005
[375]	validation_0-merror:0.052039
[376]	validation_0-merror:0.052059
[377]	validation_0-merror:0.052052
[378]	validation_0-merror:0.052032
[379]	validation_0-merror:0.052019
[380]	validation_0-merror:0.051978
[381]	validation_0-merror:0.051998
[382]	validation_0-merror:0.052025
[383]	validation_0-merror:0.052012
[384]	validation_0-merror:0.051992
[385]	validation_0-merror:0.052019
[386]	validation_0-merror:0.052052
[387]	validation_0-merror:0.052052
[388]	validation_0-merror:0.051958
[389]	validation_0-merror:0.051931
[390]	validation_0-merror:0.051911
[391]	validation_0-merror:0.051857
[392]	validation_0-merror:0.051945
[393]	validation_0-merror:0.051978
[394]	validation_0-merror:0.051931
[395]	validation_0-merror:0.051985
[396]	validation_0-m

In [240]:
submit_xgb = []
for line in cv_pred:
    submit_xgb.append(np.argmax(np.bincount(line)))

In [241]:
xgb_oof_train.astype(int)

array([6, 3, 4, ..., 3, 0, 8])

In [242]:
trpre_xgb = le.inverse_transform(xgb_oof_train.astype(int))
df_train = train_id[['user_id']]
df_train['predict_service'] = trpre_xgb

In [243]:
tr_label = train[['user_id', 'current_service']]

In [147]:
submit_xgb = le.inverse_transform(submit_xgb)
df_test = test_id[['user_id']]
df_test['predict_service'] = submit_xgb

In [169]:
df_test.to_csv('xgb_vote_0.743.csv', index=None)

## 存储临时数据

In [166]:
submit_xgb = np.array(submit_xgb)
np.save('submit_xgb_0.743.pny', submit_xgb)

In [168]:
np.save('xgb_oof_train_0.743.pny', xgb_oof_train)
# np.save('lgb_oof_train.pny', lgb_oof_train)

In [40]:
submit_lgb = np.array(submit_lgb)
np.save('submit_lgb.pny', submit_lgb)

In [244]:
score(tr_label, df_train)

0.9592992045288177 99999825
0.9976285410826529 90063345
0.9964324125809921 90109916
0.944666371315714 89950166
0.9676266076865978 89950168
0.9079679569497952 89950167
0.9715852442671985 90155946
0.8714209766841345 99999828
0.86108567909044 99999826
0.8289593598912556 99999827
0.8079105846199598 99999830


0.8454941158991052

## 第二层XGB预测

In [36]:
multi_train = np.concatenate((lgb_oof_train.reshape(-1, 1), xgb_oof_train.reshape(-1, 1)), axis=1)
multi_test = np.concatenate((np.array(submit_lgb).reshape(-1, 1), np.array(submit_xgb).reshape(-1, 1)), axis=1)

In [37]:
def XGB2():
    clf = xgb.XGBClassifier(max_depth=10, learning_rate=0.1,
                            n_estimators=300, silent=True,
                            objective="multi:softmax",
                            nthread=4, gamma=0,
                            max_delta_step=0, subsample=1, colsample_bytree=0.9, colsample_bylevel=0.9,
                            reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
                            base_score=0.5, seed=2018, missing=None,num_class=11)
    return clf

In [44]:
clf = XGB2()
clf.fit(multi_train, train_y_le)
submit = clf.predict(multi_test)

In [45]:
submit_multi = le.inverse_transform(submit)
df_test = test_id[['user_id']]
df_test['predict_service'] = submit_multi

In [ ]:
df_test.to_csv('multi_vote.csv', index=None)

后面的都不用管

## GBDT+LR

In [ ]:
# LR + GBDT
#得到叶节点编号 Feature Transformation
gbdt_feats_vals = clf.predict(combineDf, pred_leaf=True)
gbdt_columns = ["gbdt_leaf_indices_" + str(i) for i in range(0, gbdt_feats_vals.shape[1])]
combineDf = pd.concat([combineDf, pd.DataFrame(data=gbdt_feats_vals, index=range(0, gbdt_feats_vals.shape[0]),columns=gbdt_columns)], axis=1)

# onehotencoder(gbdt_feats)
origin_columns = combineDf.columns
for col in gbdt_columns:
    combineDf = pd.concat([combineDf, pd.get_dummies(combineDf[col], prefix=col)],axis=1)
gbdt_onehot_feats = [col for col in combineDf.columns if col not in origin_columns]
lr_gbdt_feats = lgb_feats+ gbdt_onehot_feats
lr_gbdt_model = LogisticRegression(penalty='l2', C=1)
lr_gbdt_model.fit(train[lr_gbdt_feats],current_service)

## GridSearchCV选择参数

In [80]:
train_tra_x = train_x[1:int(4/5*len(train_x))]
train_val_x = train_x[int(4/5*len(train_x)):]

In [81]:
train_tra_y = train_y_le[1:int(4/5*len(train_x))]
train_val_y = train_y_le[int(4/5*len(train_x)):]

In [73]:
from sklearn.model_selection import GridSearchCV

model_lgb = lgb.LGBMClassifier(objective='multiclass', learning_rate=0.05, n_estimators=2000, reg_alpha=0.1, reg_lambda=0.2)
param = {'max_depth':range(7, 9), 'num_leaves':range(100, 130, 20)}
search = GridSearchCV(estimator=model_lgb, param_grid=param, cv=5, verbose=1, n_jobs=2)

In [74]:
search.fit(train4_x, train4_y_le)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed: 619.7min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.05, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=2000, n_jobs=-1, num_leaves=31,
        objective='multiclass', random_state=None, reg_alpha=0.1,
        reg_lambda=0.2, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'max_depth': range(7, 9), 'num_leaves': range(100, 130, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [75]:
search.best_params_

{'max_depth': 8, 'num_leaves': 120}

## cv实验参数的合理性

In [20]:
column_str = ['service_type', 'is_mix_service', 'many_over_bill', 'contract_type', 'is_promise_low_consume', 'net_service', 'gender']
train_data = lgb.Dataset(train_tra_x, label=train_tra_y, feature_name=list(train_tra_x.columns), categorical_feature=column_str)
validate_data = lgb.Dataset(train_val_x, label=train_val_y, feature_name=list(train_tra_x.columns), categorical_feature=column_str)
train_all = lgb.Dataset(train_x, train_y_le, feature_name=list(train_tra_x.columns), categorical_feature=column_str)

In [21]:
params4 = {'learning_rate':0.1, 
          'lambda_l1':0.18,
          'lambda_l2':0.23,
          'max_depth':7,
          'num_leaves':100, 
           'n_estimators':3000, 
          'objective':'multiclass',
          'num_class':11
          }

In [22]:
cv_result = lgb.cv(params4, train_all4, nfold=5, verbose_eval=50, early_stopping_rounds=50)

KeyboardInterrupt: 

In [145]:
# print('best n_estinators:', len(cv_result['multi_logloss-mean']))

best n_estinators: 700


0.16202

In [34]:
lgb4 = lgb.train(params4, train_data4, valid_sets=[validate_data4])
# lgb4 = lgb.train(params4, train_all4)

[1]	valid_0's multi_logloss: 2.18645
[2]	valid_0's multi_logloss: 2.02188
[3]	valid_0's multi_logloss: 1.88648
[4]	valid_0's multi_logloss: 1.7709
[5]	valid_0's multi_logloss: 1.67019
[6]	valid_0's multi_logloss: 1.58073
[7]	valid_0's multi_logloss: 1.50067
[8]	valid_0's multi_logloss: 1.42875
[9]	valid_0's multi_logloss: 1.36316
[10]	valid_0's multi_logloss: 1.30311
[11]	valid_0's multi_logloss: 1.24737
[12]	valid_0's multi_logloss: 1.19622
[13]	valid_0's multi_logloss: 1.14849
[14]	valid_0's multi_logloss: 1.10405
[15]	valid_0's multi_logloss: 1.06263
[16]	valid_0's multi_logloss: 1.02373
[17]	valid_0's multi_logloss: 0.986771
[18]	valid_0's multi_logloss: 0.952232
[19]	valid_0's multi_logloss: 0.91976
[20]	valid_0's multi_logloss: 0.889169
[21]	valid_0's multi_logloss: 0.860363
[22]	valid_0's multi_logloss: 0.83319
[23]	valid_0's multi_logloss: 0.807429
[24]	valid_0's multi_logloss: 0.783102
[25]	valid_0's multi_logloss: 0.760027
[26]	valid_0's multi_logloss: 0.738209
[27]	valid_0's

[210]	valid_0's multi_logloss: 0.211576
[211]	valid_0's multi_logloss: 0.211335
[212]	valid_0's multi_logloss: 0.211087
[213]	valid_0's multi_logloss: 0.210867
[214]	valid_0's multi_logloss: 0.210612
[215]	valid_0's multi_logloss: 0.210353
[216]	valid_0's multi_logloss: 0.210114
[217]	valid_0's multi_logloss: 0.209949
[218]	valid_0's multi_logloss: 0.209694
[219]	valid_0's multi_logloss: 0.209505
[220]	valid_0's multi_logloss: 0.209324
[221]	valid_0's multi_logloss: 0.209059
[222]	valid_0's multi_logloss: 0.20885
[223]	valid_0's multi_logloss: 0.208659
[224]	valid_0's multi_logloss: 0.208473
[225]	valid_0's multi_logloss: 0.20824
[226]	valid_0's multi_logloss: 0.208052
[227]	valid_0's multi_logloss: 0.207787
[228]	valid_0's multi_logloss: 0.207548
[229]	valid_0's multi_logloss: 0.207347
[230]	valid_0's multi_logloss: 0.207161
[231]	valid_0's multi_logloss: 0.207004
[232]	valid_0's multi_logloss: 0.206784
[233]	valid_0's multi_logloss: 0.206524
[234]	valid_0's multi_logloss: 0.206273
[2

[416]	valid_0's multi_logloss: 0.185364
[417]	valid_0's multi_logloss: 0.185281
[418]	valid_0's multi_logloss: 0.185223
[419]	valid_0's multi_logloss: 0.185138
[420]	valid_0's multi_logloss: 0.185085
[421]	valid_0's multi_logloss: 0.185015
[422]	valid_0's multi_logloss: 0.184932
[423]	valid_0's multi_logloss: 0.184868
[424]	valid_0's multi_logloss: 0.184808
[425]	valid_0's multi_logloss: 0.184746
[426]	valid_0's multi_logloss: 0.184698
[427]	valid_0's multi_logloss: 0.184642
[428]	valid_0's multi_logloss: 0.184584
[429]	valid_0's multi_logloss: 0.184503
[430]	valid_0's multi_logloss: 0.184439
[431]	valid_0's multi_logloss: 0.184386
[432]	valid_0's multi_logloss: 0.18433
[433]	valid_0's multi_logloss: 0.18428
[434]	valid_0's multi_logloss: 0.184207
[435]	valid_0's multi_logloss: 0.184149
[436]	valid_0's multi_logloss: 0.184078
[437]	valid_0's multi_logloss: 0.184016
[438]	valid_0's multi_logloss: 0.183966
[439]	valid_0's multi_logloss: 0.18391
[440]	valid_0's multi_logloss: 0.183853
[44

[622]	valid_0's multi_logloss: 0.175234
[623]	valid_0's multi_logloss: 0.175206
[624]	valid_0's multi_logloss: 0.175162
[625]	valid_0's multi_logloss: 0.175134
[626]	valid_0's multi_logloss: 0.175092
[627]	valid_0's multi_logloss: 0.175048
[628]	valid_0's multi_logloss: 0.175008
[629]	valid_0's multi_logloss: 0.174966
[630]	valid_0's multi_logloss: 0.174936
[631]	valid_0's multi_logloss: 0.174907
[632]	valid_0's multi_logloss: 0.174878
[633]	valid_0's multi_logloss: 0.174863
[634]	valid_0's multi_logloss: 0.174831
[635]	valid_0's multi_logloss: 0.174797
[636]	valid_0's multi_logloss: 0.174754
[637]	valid_0's multi_logloss: 0.174713
[638]	valid_0's multi_logloss: 0.174681
[639]	valid_0's multi_logloss: 0.174646
[640]	valid_0's multi_logloss: 0.174614
[641]	valid_0's multi_logloss: 0.174579
[642]	valid_0's multi_logloss: 0.174541
[643]	valid_0's multi_logloss: 0.174522
[644]	valid_0's multi_logloss: 0.17449
[645]	valid_0's multi_logloss: 0.174455
[646]	valid_0's multi_logloss: 0.174429
[

[828]	valid_0's multi_logloss: 0.169318
[829]	valid_0's multi_logloss: 0.16929
[830]	valid_0's multi_logloss: 0.169276
[831]	valid_0's multi_logloss: 0.169266
[832]	valid_0's multi_logloss: 0.169218
[833]	valid_0's multi_logloss: 0.169202
[834]	valid_0's multi_logloss: 0.169182
[835]	valid_0's multi_logloss: 0.169172
[836]	valid_0's multi_logloss: 0.169158
[837]	valid_0's multi_logloss: 0.169136
[838]	valid_0's multi_logloss: 0.169109
[839]	valid_0's multi_logloss: 0.169084
[840]	valid_0's multi_logloss: 0.169056
[841]	valid_0's multi_logloss: 0.169038
[842]	valid_0's multi_logloss: 0.169017
[843]	valid_0's multi_logloss: 0.168999
[844]	valid_0's multi_logloss: 0.168975
[845]	valid_0's multi_logloss: 0.168961
[846]	valid_0's multi_logloss: 0.168937
[847]	valid_0's multi_logloss: 0.168909
[848]	valid_0's multi_logloss: 0.16889
[849]	valid_0's multi_logloss: 0.168857
[850]	valid_0's multi_logloss: 0.168836
[851]	valid_0's multi_logloss: 0.168814
[852]	valid_0's multi_logloss: 0.168788
[8

...

[1234]	valid_0's multi_logloss: 0.162319
[1235]	valid_0's multi_logloss: 0.162307
[1236]	valid_0's multi_logloss: 0.162293
[1237]	valid_0's multi_logloss: 0.162284
[1238]	valid_0's multi_logloss: 0.162267
[1239]	valid_0's multi_logloss: 0.162255
[1240]	valid_0's multi_logloss: 0.162241
[1241]	valid_0's multi_logloss: 0.162235
[1242]	valid_0's multi_logloss: 0.162223
[1243]	valid_0's multi_logloss: 0.162207
[1244]	valid_0's multi_logloss: 0.162192
[1245]	valid_0's multi_logloss: 0.162181
[1246]	valid_0's multi_logloss: 0.162169
[1247]	valid_0's multi_logloss: 0.162142
[1248]	valid_0's multi_logloss: 0.162126
[1249]	valid_0's multi_logloss: 0.162103
[1250]	valid_0's multi_logloss: 0.162087
[1251]	valid_0's multi_logloss: 0.162074
[1252]	valid_0's multi_logloss: 0.162049
[1253]	valid_0's multi_logloss: 0.162035
[1254]	valid_0's multi_logloss: 0.162022
[1255]	valid_0's multi_logloss: 0.162014
[1256]	valid_0's multi_logloss: 0.162005
[1257]	valid_0's multi_logloss: 0.16199
[1258]	valid_0's 

[1435]	valid_0's multi_logloss: 0.160341
[1436]	valid_0's multi_logloss: 0.160337
[1437]	valid_0's multi_logloss: 0.160329
[1438]	valid_0's multi_logloss: 0.160318
[1439]	valid_0's multi_logloss: 0.160308
[1440]	valid_0's multi_logloss: 0.160302
[1441]	valid_0's multi_logloss: 0.160294
[1442]	valid_0's multi_logloss: 0.160289
[1443]	valid_0's multi_logloss: 0.160285
[1444]	valid_0's multi_logloss: 0.160277
[1445]	valid_0's multi_logloss: 0.160264
[1446]	valid_0's multi_logloss: 0.160252
[1447]	valid_0's multi_logloss: 0.160248
[1448]	valid_0's multi_logloss: 0.160238
[1449]	valid_0's multi_logloss: 0.160237
[1450]	valid_0's multi_logloss: 0.160225
[1451]	valid_0's multi_logloss: 0.160213
[1452]	valid_0's multi_logloss: 0.160201
[1453]	valid_0's multi_logloss: 0.160191
[1454]	valid_0's multi_logloss: 0.160184
[1455]	valid_0's multi_logloss: 0.160177
[1456]	valid_0's multi_logloss: 0.16017
[1457]	valid_0's multi_logloss: 0.16017
[1458]	valid_0's multi_logloss: 0.160163
[1459]	valid_0's m

[1636]	valid_0's multi_logloss: 0.159272
[1637]	valid_0's multi_logloss: 0.159269
[1638]	valid_0's multi_logloss: 0.159267
[1639]	valid_0's multi_logloss: 0.159265
[1640]	valid_0's multi_logloss: 0.159264
[1641]	valid_0's multi_logloss: 0.159265
[1642]	valid_0's multi_logloss: 0.159264
[1643]	valid_0's multi_logloss: 0.159262
[1644]	valid_0's multi_logloss: 0.15926
[1645]	valid_0's multi_logloss: 0.15926
[1646]	valid_0's multi_logloss: 0.159257
[1647]	valid_0's multi_logloss: 0.159256
[1648]	valid_0's multi_logloss: 0.159252
[1649]	valid_0's multi_logloss: 0.159254
[1650]	valid_0's multi_logloss: 0.159253
[1651]	valid_0's multi_logloss: 0.159255
[1652]	valid_0's multi_logloss: 0.159255
[1653]	valid_0's multi_logloss: 0.159253
[1654]	valid_0's multi_logloss: 0.159249
[1655]	valid_0's multi_logloss: 0.159248
[1656]	valid_0's multi_logloss: 0.159248
[1657]	valid_0's multi_logloss: 0.159249
[1658]	valid_0's multi_logloss: 0.159246
[1659]	valid_0's multi_logloss: 0.159245
[1660]	valid_0's m

[1837]	valid_0's multi_logloss: 0.159121
[1838]	valid_0's multi_logloss: 0.159121
[1839]	valid_0's multi_logloss: 0.159121
[1840]	valid_0's multi_logloss: 0.159121
[1841]	valid_0's multi_logloss: 0.159121
[1842]	valid_0's multi_logloss: 0.159121
[1843]	valid_0's multi_logloss: 0.159121
[1844]	valid_0's multi_logloss: 0.159121
[1845]	valid_0's multi_logloss: 0.159121
[1846]	valid_0's multi_logloss: 0.159121
[1847]	valid_0's multi_logloss: 0.159121
[1848]	valid_0's multi_logloss: 0.159121
[1849]	valid_0's multi_logloss: 0.159121
[1850]	valid_0's multi_logloss: 0.159121
[1851]	valid_0's multi_logloss: 0.159121
[1852]	valid_0's multi_logloss: 0.159121
[1853]	valid_0's multi_logloss: 0.159121
[1854]	valid_0's multi_logloss: 0.159121
[1855]	valid_0's multi_logloss: 0.159121
[1856]	valid_0's multi_logloss: 0.159121
[1857]	valid_0's multi_logloss: 0.159121
[1858]	valid_0's multi_logloss: 0.159121
[1859]	valid_0's multi_logloss: 0.159121
[1860]	valid_0's multi_logloss: 0.159121
[1861]	valid_0's

KeyboardInterrupt: 

In [49]:
val_lgb4 = lgb4.predict(train4_val_x)
val_lgb4 = [list(x).index(max(x)) for x in val_lgb4]

In [50]:
validation4 = train4_id[int(4/5*len(train4_x_stand)):]
validation4['predict_service'] = le4.inverse_transform(val_lgb4)

In [51]:
val_y4 = train4_id[int(4/5*len(train4_x_stand)):]
val_y4['current_service'] = le4.inverse_transform(train4_val_y)

use_situation 上升

pay caller_time 都没什么用 age_like会降分

total_fee: 0.84129756 0.7347

online_mix:0.83459 0.7340

baseline_7_100: 0.829 线上0.738

In [53]:
import matplotlib.pyplot as plt

print(pd.DataFrame({
        'column': train4_feature,
        'importance': lgb4.feature_importance(),
    }).sort_values(by='importance'))

# plt.plot(sorted(lgb4.feature_importance()))

                    column  importance
23    former_complaint_num          92
12             net_service         169
13               pay_times         398
11  is_promise_low_consume         467
22         complaint_level         494
1           is_mix_service         572
24    former_complaint_fee         658
20                  gender         851
0             service_type         963
17       local_caller_time        1276
21                     age        1812
8           many_over_bill        2016
14                 pay_num        2050
16    local_trafffic_month        3053
10           contract_time        3585
2              online_time        3937
18    service1_caller_time        4068
6              4_total_fee        4334
7            month_traffic        5076
4              2_total_fee        5257
9            contract_type        5457
19    service2_caller_time        6945
5              3_total_fee        7003
15      last_month_traffic        7700
3              1_total_fe

# 测试集

In [39]:
test4 = pd.read_csv('test4_featured.csv')
# test1 = pd.read_csv('test1_featured.csv')

In [41]:
test4_id = test4[['user_id']]
test4_x = test4.drop(['user_id'], axis=1)

In [42]:
test4_x_stand = test4_x

In [43]:
pre_test4 = lgb4.predict(test4_x_stand)

In [44]:
pre_test4 = [list(x).index(max(x)) for x in pre_test4]

In [45]:
pre_test4 = le4.inverse_transform(pre_test4)

In [46]:
predict4 = test4_id
predict4['predict_service'] = pre_test4

In [47]:
predict4.to_csv('lgb_all1010_8_150.csv', index=None)

In [154]:
def score(tr, te):
    labels = tr['current_service'].unique()
    f1_score = 0
    for label in labels:
        tp = 0
        fp = 0
        fn = 0
        sub_tr = tr[tr['current_service'] == label]['current_service']
        index_tr = tr[tr['current_service'] == label].index.tolist()
        sub_te = te['predict_service'][index_tr]
        for la in sub_te.values:
            if la == label:
                tp = tp + 1
            else:
                fn = fn + 1
        index_te = te[te['predict_service'] == label].index.tolist()
        fp_list = set(index_te).difference(index_tr)
        fp = len(fp_list)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = f1_score + (2*precision*recall)/(precision+recall)
        print((2*precision*recall)/(precision+recall), label)
    final_score = (1/(len(labels))*f1_score)**2
    
    return final_score

In [119]:
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali['current_service'].unique()
    preds = np.argmax(preds.reshape(11, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

In [149]:
def score(tr, te):
    labels = tr['current_service'].unique()
    f1_score = 0
    for label in labels:
        tp = 0
        fp = 0
        fn = 0
        sub_tr = tr[tr['current_service'] == label]['current_service']
        index_tr = tr[tr['current_service'] == label].index.tolist()
        sub_te = te['predict_service'][index_tr]
        for la in sub_te.values:
            if la == label:
                tp = tp + 1
            else:
                fn = fn + 1
        index_te = te[te['predict_service'] == label].index.tolist()
        fp_list = set(index_te).difference(index_tr)
        fp = len(fp_list)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = f1_score + (2*precision*recall)/(precision+recall)
    final_score = (1/(len(labels))*f1_score)**2
    
    return final_score

In [316]:
a = pd.read_csv('base_v1021_other.csv')
b = pd.read_csv('xgb_base_0.744.csv')

In [318]:
b = b.rename(columns={'predict_service':'current_service'})

In [322]:
a = a.rename(columns={'predict':'predict_service'})

In [323]:
score(b, a)

0.9984820127131435 90063345
0.8754381864921711 99999827
0.9675936010068007 89950167
0.9476171267216043 89950166
0.8328759604829856 99999830
0.9266418124462289 99999828
0.9963114547016321 90109916
0.9230908269317668 99999826
0.9701076125946593 89950168
0.972011751971548 99999825
0.977217366384869 90155946


0.8917175494934166